In [ ]:
"""
Purpose: Run the Soma Finding
Algorithm for all cells in the 
multi soma proofreading



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-07 01:20:25,438 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-07 01:20:25,440 - settings - Setting database.user to celiib
INFO - 2021-01-07 01:20:25,441 - settings - Setting database.password to newceliipass
INFO - 2021-01-07 01:20:25,445 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-07 01:20:25,445 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-07 01:20:25,456 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-07 01:20:25,678 - settings - Setting enable_python_native_blobs to True


In [2]:
test_mode = True

In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-07 01:20:25,857 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-07 01:20:25,858 - settings - Setting database.user to celiib
INFO - 2021-01-07 01:20:25,859 - settings - Setting database.password to newceliipass
INFO - 2021-01-07 01:20:25,862 - settings - Setting enable_python_native_blobs to True


Sleeping 97 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-07 01:20:26,086 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [4]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 3834


In [7]:
#so that it will have the adapter defined
from datajoint_utils import *

In [15]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 3


import soma_extraction_utils as sm
@schema
class BaylorSegmentCentroid(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ver : tinyint unsigned #the version number of the materializaiton
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    mesh: <somas>  #datajoint adapter to get the somas mesh objects
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    max_hole_length=NULL      : double                    #euclidean distance of the maximum hole size
    run_time=NULL : double                   # the amount of time to run (seconds)

    """
    key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei()) & minnie.MultiSomaProofread2.proj()

     

    def make(self,key):
        """
        Pseudocode: 
        1) Compute all of the
        2) Save the mesh as an h5 py file
        3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        4) Insert
        
        
        """
        
        #get the mesh data
        print(f"\n\n\n---- Working on {key['segment_id']} ----")
        print(key)
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

        segment_id = key["segment_id"]

        (total_soma_list, 
         run_time, 
         total_soma_list_sdf) = sm.extract_soma_center(
                            segment_id,
                            current_mesh_verts,
                            current_mesh_faces,
        )
        
        
        print(f"Run time was {run_time} \n    total_soma_list = {total_soma_list}"
             f"\n    with sdf values = {total_soma_list_sdf}")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            

            returned_file_path = tu.write_h5_file(
                                                vertices=np.array([]),
                                                  faces=np.array([]),
                                                  segment_id=segment_id,
                                                  filename = f'{segment_id}_0.h5',
                                                    filepath=str(du.get_somas_path())
                                                 )

            
            
            insert_dict = dict(key,
                              soma_index=0,
                               ver=current_version,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               #distance_from_prediction=None,
                               #prediction_matching_index = None,
                               n_vertices=0,
                               n_faces=0,
                               mesh=returned_file_path,
                               multiplicity=0,
                               sdf = None,
                               max_side_ratio = None,
                               bbox_volume_ratio = None,
                               max_hole_length=None,
                               run_time=run_time
                              )
            
            #raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found, get the volume and side length checks
        max_side_ratio =  [np.max(sm.side_length_ratios(m)) for m in total_soma_list]
        bbox_volume_ratio =  [sm.soma_volume_ratio(m) for m in total_soma_list]
        dicts_to_insert = []


        for i,(current_soma,soma_sdf,sz_ratio,vol_ratio) in enumerate(zip(total_soma_list,total_soma_list_sdf,max_side_ratio,bbox_volume_ratio)):
            print("Trying to write off file")
            """ Currently don't need to export the meshes
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            """
            auto_prediction_center = np.mean(current_soma.vertices,axis=0) / np.array([4,4,40])
            auto_prediction_center = auto_prediction_center.astype("int")
            print(f"Predicted Coordinates are {auto_prediction_center}")
            max_hole_length = tu.largest_hole_length(current_soma)
            
            returned_file_path = tu.write_h5_file(
                                            vertices=current_soma.vertices,
                                              faces=current_soma.faces,
                                              segment_id=segment_id,
                                              filename = f'{segment_id}_{i}.h5',
                                                filepath=str(du.get_somas_path())
                                             )



            insert_dict = dict(key,
                              soma_index=i+1,
                               ver=current_version,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               n_vertices = len(current_soma.vertices),
                               n_faces = len(current_soma.faces),
                               mesh=returned_file_path,
                               multiplicity=len(total_soma_list),
                               sdf = np.round(soma_sdf,3),
                               max_side_ratio = np.round(sz_ratio,3),
                               bbox_volume_ratio = np.round(vol_ratio,3),
                               max_hole_length = np.round(max_hole_length,3),
                               run_time=np.round(run_time,4)
                              )



            dicts_to_insert.append(insert_dict)
        self.insert(dicts_to_insert,skip_duplicates=True)

# Running the Populate

In [16]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table
#curr_table.delete()
#curr_table.delete()
#curr_table.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [ ]:
# import pandas as pd
# key_hash,error_message = curr_table.fetch("key_hash","error_message")

# df = pd.DataFrame.from_dict([dict(key_hash = k,error_message = m) for k,m in zip(key_hash,error_message)])
# df
# #df.columns = ["error","key_hash"]
# key_hashes_to_delete = df[df["error_message"].str.contains("OSError")]["key_hash"].to_numpy()

# (curr_table & [dict(key_hash=k) for k in key_hashes_to_delete]).delete()

In [ ]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=True)
else:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")

INFO - 2021-01-07 01:29:17,643 - autopopulate - Found 58 keys to populate
INFO - 2021-01-07 01:29:17,651 - connection - Transaction started
INFO - 2021-01-07 01:29:17,654 - autopopulate - Populating: {'segment_id': 864691134947393276, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on 864691134947393276 ----
{'segment_id': 864691134947393276, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 3834 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_17310.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_17310_fill_holes.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/fill_holes_9919.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript c

/meshAfterParty/trimesh_utils.py:2565: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(289326, 3), faces.shape=(524492, 3))>, <trimesh.Trimesh(vertices.shape=(9161, 3), faces.shape=(15086, 3))>, <trimesh.Trimesh(vertices.shape=(3115, 3), faces.shape=(5177, 3))>, <trimesh.Trimesh(vertices.shape=(2397, 3), faces.shape=(3931, 3))>, <trimesh.Trimesh(vertices.shape=(1399, 3), faces.shape=(2210, 3))>, <trimesh.Trimesh(vertices.shape=(1355, 3), faces.shape=(2032, 3))>, <trimesh.Trimesh(vertices.shape=(1314, 3), faces.shape=(2138, 3))>, <trimesh.Trimesh(vertices.shape=(1286, 3), faces.shape=(2089, 3))>, <trimesh.Trimesh(vertices.shape=(1280, 3), faces.shape=(1975, 3))>, <trimesh.Trimesh(vertices.shape=(1017, 3), faces.shape=(1626, 3))>, <trimesh.Trimesh(vertices.shape=(1002, 3), faces.shape=(1648, 3))>, <trimesh.Trimesh(vertices.shape=(867, 3), faces.shape=(1264, 3))>, <trimesh.Trimesh(vertices.shape=(850, 3), faces.shape=(1290, 3))>, <trimesh.Trimesh(vertices.shape=(753, 3), faces.shape=(1229, 3))>, <trime

WARNING - 2021-01-07 01:33:38,591 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,480 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,485 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,603 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,694 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,732 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,852 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:39,924 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:40,024 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:40,057 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 01:33:40,281 - base - face_normals all zero, ignoring!


...

/meshAfterParty/trimesh_utils.py:2565: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(22982, 3), faces.shape=(56929, 3))>, <trimesh.Trimesh(vertices.shape=(19806, 3), faces.shape=(48682, 3))>, <trimesh.Trimesh(vertices.shape=(757, 3), faces.shape=(2127, 3))>]
xvfb-run -n 3834 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135740250731/neuron_864691135740250731.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135740250731/neuron_864691135740250731_decimated.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/864691135740250731/decimation_meshlab_25555532.mls


WARNING - 2021-01-07 05:47:27,618 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,636 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,812 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,815 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,870 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,872 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,873 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,879 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,891 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,892 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,900 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,900 - base - face_normals all zero, ignoring!
WARNING - 2021-01-07 05:47:27,901 - base - face_normals all zero, ignoring!
WARNING - 20

Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(183390, 3), faces.shape=(361918, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(183390, 3), faces.shape=(361918, 3))>
remove_inside_pieces requested 
xvfb-run -n 3834 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_28773.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/neuron_28773_fill_holes.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/fill_holes_278784.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that do


-------Splits after inner decimation len = 1--------

done exporting decimated mesh: neuron_864691135740250731_decimated_largest_piece_poisson_largest_inner.off
largest_mesh_path_inner_decimated_clean = <trimesh.Trimesh(vertices.shape=(2037, 3), faces.shape=(4070, 3))>

1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0007085800170898438
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2, path_and_filename = /notebooks/Auto_Proofreading/Preprocessing_Pipeline/temp/86469113574025073101_fixed 
1
Finished CGAL segmentation algorithm: 0.3112468719482422
2) Finished: Generating CGAL segmentation for neuron: 0.3972935676574707
3) Staring: Generating Graph Structure and Identifying Soma using soma size threshold  = 3000
my_list_keys = [0, 1, 2, 3, 4, 5, 6]
soma_index = -1
3) Finished: Generatin